In [0]:
import pandas as pd
import numpy as np

from sklearn.ensemble import RandomForestRegressor


!pip install category_encoders
from category_encoders.one_hot import OneHotEncoder


In [0]:
#train = pd.read_csv("train.csv")
train = pd.read_csv("https://raw.githubusercontent.com/nilberthsouza/Titanic-Kaggle/master/train.csv")
#test  = pd.read_csv("test.csv"
test  = pd.read_csv("https://raw.githubusercontent.com/nilberthsouza/Titanic-Kaggle/master/test.csv")

train.head()

In [75]:
print(train.Age.mean(skipna=True))
print(test.Age.mean(skipna=True))
print(train.shape[1])
print(test.shape[1])
print(train.columns)
print(test.columns)

29.69911764705882
30.272590361445783
12
11
Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')
Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch',
       'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


In [76]:
resume = train.Survived.copy()
print(resume)

0      0
1      1
2      1
3      1
4      0
      ..
886    0
887    1
888    0
889    1
890    0
Name: Survived, Length: 891, dtype: int64


In [0]:
train.drop(['Survived'],axis=1,inplace=True)
passengerId = test['PassengerId']

train_index = train.shape[0]
test_index = test.shape[0]

full = pd.concat(objs=[train,test],axis=0).reset_index(drop=True)
full.head()

In [0]:
full.drop(['PassengerId','Name','Cabin','Ticket'],axis=1,inplace=True)
full.head()

In [0]:
full.isnull().sum()

In [0]:

from category_encoders.ordinal import OrdinalEncoder
enc = OrdinalEncoder(cols=['Embarked'])
enc.fit(full)
full = enc.transform(full)


In [0]:
modeAge = int(full['Age'].mode())
full['Age'].fillna(modeAge,inplace=True)

avgFare = full['Fare'].median()
full['Fare'].fillna(avgFare,inplace=True)

mcEmbarked = int(full['Embarked'].mode())
full['Embarked'].fillna(mcEmbarked,inplace=True)

full.isnull().sum()

In [0]:
enc = OneHotEncoder(cols=['Pclass','Sex'], use_cat_names=True)
enc.fit(full)
full = enc.transform(full)

In [0]:
train = full.iloc[:train_index]
test = full.iloc[:test_index]

In [0]:
test.isna().sum()

In [0]:
mdl = RandomForestRegressor(n_jobs=1, n_estimators=100, random_state=22)
mdl.fit(train,resume)


In [0]:
P_ohe = mdl.predict(test)

In [0]:
P_ohe[:5]

In [0]:
from sklearn.metrics import mean_absolute_error


In [0]:
submission = pd.DataFrame({"PassengerId": passengerId,
                           "Survived":P_ohe
                          })

submission.to_csv('./submission_py.csv',index=False)